# Normalización



In [5]:
import nltk
import re
import string
from pprint import pprint
corpus = ["The brown fox wasn't that quick and he couldn't win the race",
          "Hey that's a great deal! I just bought a phone for $199",
          "@@You'll (learn) a **lot** in the book. Python is an amazing language!@@"]

## Limpieza del texto

Los datos a utilizar pueden contener caracteres o tokens innecesarios, por lo que deben ser eliminados antes de realizar alguna operación como la tokenización o normalización. Por ejemplo, etiquetas HTML, XML o JSON. 

Existen distintas formas de limpiar el texto con funciones como `clean_html()` de `nltk` o con la librería `BeautifulSoup`. También se pueden utilizar expresiones regulares, xpath o la librería lxml.



In [11]:
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return word_tokens

In [7]:
token_list = [tokenize_text(text) for text in corpus] 
pprint(token_list)

[[['The',
   'brown',
   'fox',
   'was',
   "n't",
   'that',
   'quick',
   'and',
   'he',
   'could',
   "n't",
   'win',
   'the',
   'race']],
 [['Hey', 'that', "'s", 'a', 'great', 'deal', '!'],
  ['I', 'just', 'bought', 'a', 'phone', 'for', '$', '199']],
 [['@',
   '@',
   'You',
   "'ll",
   '(',
   'learn',
   ')',
   'a',
   '*',
   '*',
   'lot',
   '*',
   '*',
   'in',
   'the',
   'book',
   '.'],
  ['Python', 'is', 'an', 'amazing', 'language', '!'],
  ['@', '@']]]


### Eliminación caracteres especiales

Una tarea previa a la normalización del texto es eliminar los caracteres especiales. Por ejemplo símbolos especiales, signos de puntuación. Este paso regularmente es ejecutado antes o después de la tokenización. La razón de este paso es eliminar elementos que no tienen significado cuando se analiza el texto, o se extraen características o información basada en NLP o ML. 

El siguiente código muestra la eleminición de caracteres especiales después de la tokenización:

In [8]:
def remove_characters_after_tokenization(tokens):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation))) 
    filtered_tokens = filter(None, [pattern.sub('', token) for token in tokens]) 
    return filtered_tokens

In [11]:
filtered_list_1 =  [filter(None,[remove_characters_after_tokenization(tokens) for tokens in sentence_tokens]) for sentence_tokens in token_list]
pprint(filtered_list_1)

El atributo `string.puntuation` consiste en todos los símbolos y caracteres especiales, y crear patrones de expresiones regulares de estos. La función `filter` ayuda a remover los tokens vacios obtenidos después de revomer token de caracteres especiales utilizando el método `reg sub`.


In [19]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]' # add other characters here to remove them
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]' # only extract alpha-numeric characters
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

In [17]:
filtered_list_2 = [remove_characters_before_tokenization(sentence) for sentence in corpus]
print(filtered_list_2)

['The brown fox wasnt that quick and he couldnt win the race', 'Hey thats a great deal I just bought a phone for 199', 'Youll learn a lot in the book Python is an amazing language']


In [20]:
cleaned_corpus = [remove_characters_before_tokenization(sentence, keep_apostrophes=True) for sentence in corpus]
print(cleaned_corpus)

["The brown fox wasn't that quick and he couldn't win the race", "Hey that's a great deal! I just bought a phone for 199", "You'll learn a lot in the book. Python is an amazing language!"]


## Expasión de contracciones

Las contracciones son versiónes cortas de palabras o silabas. En inglés los constracciones son creadas removiendo vocales de la palabra. Por ejemplo, is not - isn't, will not - won't. Usualmente las constracciones son evitadas en escritura formal, pero en la escritura informal son muy comunes. Existen varias formas de contracción que esta ligadas al tipo de verbo auxiliar que da una forma de contracción normal, negada u otras contracciones coloquiales especiales, algunas de ellas no implican auxiliares. 

Las contracciones implican un problema en NLP y análisis de texto, ya que contiene el caracter especial apostrofe en la palabra. Además, se tiene dos o más palabras representadas en la contracción, espo amplia las formas que se puede tokenizar una palabra. Idealmente, se puede proponer un mapeo de contracciones y sus correspondientes expansiones.

Se puede crear un archivo `contractions.py` en un diccionario de Python.

````Python
CONTRACTION_MAP = {
"isn't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
.
.
.
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
````
Se debe considera que algunas de las contracciones tienen multiplés formas. Por ejemplo, *you'll* puede indicar *you will* o *you shall*.

In [9]:
from contractions import CONTRACTION_MAP
def expand_contractions(sentence, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence

In [10]:
expanded_corpus = [expand_contractions(sentence, CONTRACTION_MAP) for sentence in cleaned_corpus]

print(expanded_corpus)

['The brown fox was not that quick and he could not win the race', 'Hey that is a great deal! I just bought a phone for 199', 'You will learn a lot in the book. Python is an amazing language!']


## Mayúsculas y minúsculas

In [11]:
print(corpus[0].lower())

the brown fox wasn't that quick and he couldn't win the race


In [12]:
print(corpus[0].upper())

THE BROWN FOX WASN'T THAT QUICK AND HE COULDN'T WIN THE RACE


## Palabras vacías o _stopwords_


In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cmillan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

En la siguiente función se utiliza la lista de _stopwords_ para el inglés, y se utiliza par filtrar todos los tokens que coinciden con una palabra vacía. 

In [17]:
def remove_stopwords(tokens):
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    return filtered_tokens

Primero se utiliza la función `tokenize_text` sobre la variable `expanded_corpus` de la sección anterior para eliminar las palabras vacías.

In [18]:
expanded_corpus_tokens = [tokenize_text(text) for text in expanded_corpus]

filtered_list_3 =  [[remove_stopwords(tokens) for tokens in sentence_tokens] for sentence_tokens in expanded_corpus_tokens]
print(filtered_list_3)

[[['The', 'brown', 'fox', 'quick', 'could', 'win', 'race']], [['Hey', 'great', 'deal', '!'], ['I', 'bought', 'phone', '199']], [['You', 'learn', 'lot', 'book', '.'], ['Python', 'amazing', 'language', '!']]]


El resultado muestra una reducción en el número de tokens en comparación con la tokenización inicial.

Se puede conocer el vocabulario de nltk de palabras vacías para el inglés.

In [20]:
print(nltk.corpus.stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Es importante considerar que las negaciones como _not_ y _no_ son eliminadas en esta caso (de la primera sentencia), y esto puede afectar la esencia del contexto de una oración en casos como el análisis de sentimientos. Por lo tanto, es necesario considerar no eliminar este tipo de palabras en este tipo de escenarios.

## Corrección de palabras

Una de los principales retos en la normalización de texto es la presencia de palabras incorrectas en el texto. La definición de incorrectas cubre desde palabras con errores ortográficos tanto palabras que contengan letras repetidas y no contribuyen mucho en su significado general.  Por ejemplo, __finally__ puede ser erróneamente escrita como **fianlly**, o alguna expresión de emoción intensa puede ser escrita como **finallllyyyyyy**. El objetivo principal en estos casos es la posible estandarización de de estas palabras de distintas formas para corregir la forma y sin perder información vital de los diferentes tokens del texto. 

In [3]:
import re 
old_word = 'finalllyyy'
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'

step = 1
while True:
    # remove one repeated character
    new_word = repeat_pattern.sub(match_substitution, old_word)
    if new_word != old_word:
        print(f'Step: {step} Word: {new_word}')
        step += 1 # update step
        # update old word to last substituted state
        old_word = new_word
        continue
    else:
        print("Final word:"), new_word
        break

Step: 1 Word: finalllyy
Step: 2 Word: finallly
Step: 3 Word: finally
Step: 4 Word: finaly
Final word:


Esto puede llevar a terminar con la palabra ***finaly***. Sin embargo, esto es  incorrecto, la palabra correcta es ***finally***. Ahora se utiliza el corpus WordNet para revisar las palabras validas en cada paso y terminar el ciclo una vez que esto es obtenido. Esto introduce la corrección semántica necesaria para el algoritmo. 

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/cmillan/nltk_data...


True

In [8]:
import re 
from nltk.corpus import wordnet

old_word = 'finalllyyy'
repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
match_substitution = r'\1\2\3'

step = 1
while True:
    # check for semantically correct word
    if wordnet.synsets(old_word):
        print("Final correct word:", old_word)
        break
    # remove one repeated character
    new_word = repeat_pattern.sub(match_substitution, old_word)
    if new_word != old_word:
        print(f'Step: {step} Word: {new_word}')
        step += 1 # update step
        # update old word to last substituted state
        old_word = new_word
        continue
    else:
        print("Final word:"), new_word
        break

Step: 1 Word: finalllyy
Step: 2 Word: finallly
Step: 3 Word: finally
Final correct word: finally


Se puede construir una mejor versión, para hacer más genérico el trato con los tokens incorrectos de las lista de tokens:

In [9]:
from nltk.corpus import wordnet
def remove_repeated_characters(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def replace(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return replace(new_word) if new_word != old_word else new_word
    correct_tokens = [replace(word) for word in tokens]
    return correct_tokens

In [12]:
sample_sentence = 'My schooool is realllllyyy amaaazingggg'
sample_sentence_tokens = tokenize_text(sample_sentence)[0]
print(sample_sentence_tokens)

['My', 'schooool', 'is', 'realllllyyy', 'amaaazingggg']


In [13]:
print(remove_repeated_characters(sample_sentence_tokens))


['My', 'school', 'is', 'really', 'amazing']


## Ortografía

Otro problema es la ortografía, existen distintas formas de lidiar con la ortografía incorrecta donde el objetivo principal es tener token con la ortografía correcta. 

In [14]:
import re, collections
def tokens(text):
    """
    Get all words from the corpus
    """
    return re.findall('[a-z]+', text.lower())

WORDS = tokens(file('big.txt').read())
WORD_COUNTS = collections.Counter(WORDS)

# top 10 words in the corpus
print(WORD_COUNTS.most_common(10))

NameError: name 'file' is not defined